### Data Preparation

In [1]:
#Get the link of each line
import requests
import re
from bs4 import BeautifulSoup
headers = {"User-Agent" : "User-Agent:Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0;"}
shmetro=requests.get('https://baike.baidu.com/item/%E4%B8%8A%E6%B5%B7%E5%9C%B0%E9%93%81/1273732',headers=headers).content.decode('utf8')
soup_metro=BeautifulSoup(shmetro,'lxml')
link_pattern=re.compile('''<a\s+data-lemmaid=\"(\d+)\"\s+href=\"([\S\s]+)\"\s+target=\"_blank\">上海地铁([\u4E00-\u9FA5\d][^站]+)</a>''')
line_link=[]
for tag in soup_metro.find_all('td'):
    if link_pattern.findall(str(tag.contents)):
        line_id,link,line=link_pattern.findall(str(tag.contents))[0]
        link='https://baike.baidu.com'+link
        if [link,line] not in line_link:
            line_link+=[[link,line]]
line_link

[['https://baike.baidu.com/item/%E4%B8%8A%E6%B5%B7%E5%9C%B0%E9%93%811%E5%8F%B7%E7%BA%BF/12092650',
  '1号线'],
 ['https://baike.baidu.com/item/%E4%B8%8A%E6%B5%B7%E5%9C%B0%E9%93%812%E5%8F%B7%E7%BA%BF/22947836',
  '2号线'],
 ['https://baike.baidu.com/item/%E4%B8%8A%E6%B5%B7%E5%9C%B0%E9%93%813%E5%8F%B7%E7%BA%BF/22947837',
  '3号线'],
 ['https://baike.baidu.com/item/%E4%B8%8A%E6%B5%B7%E5%9C%B0%E9%93%814%E5%8F%B7%E7%BA%BF/6310090',
  '4号线'],
 ['https://baike.baidu.com/item/%E4%B8%8A%E6%B5%B7%E5%9C%B0%E9%93%815%E5%8F%B7%E7%BA%BF/22947838',
  '5号线'],
 ['https://baike.baidu.com/item/%E4%B8%8A%E6%B5%B7%E5%9C%B0%E9%93%816%E5%8F%B7%E7%BA%BF/4316051',
  '6号线'],
 ['https://baike.baidu.com/item/%E4%B8%8A%E6%B5%B7%E5%9C%B0%E9%93%817%E5%8F%B7%E7%BA%BF/538923',
  '7号线'],
 ['https://baike.baidu.com/item/%E4%B8%8A%E6%B5%B7%E5%9C%B0%E9%93%818%E5%8F%B7%E7%BA%BF/12520383',
  '8号线'],
 ['https://baike.baidu.com/item/%E4%B8%8A%E6%B5%B7%E5%9C%B0%E9%93%819%E5%8F%B7%E7%BA%BF/12423895',
  '9号线'],
 ['https://baike.baidu.

In [2]:
#Get the station/station connection from line page
from collections import defaultdict
station_connection=defaultdict(list)
for line in line_link:
    line_info=requests.get(line[0],headers=headers).content.decode('utf8')
    soup_line=BeautifulSoup(line_info,'lxml')
    line_pattern=re.compile('''<[ab]\s([\S\s]+)>([\u4E00-\u9FA5\d]+)站</[ab]>''')
    line_station=[]
    line_name=soup_line.title.string[:-5]
    for tag in soup_line.find_all('td'):
        if line_pattern.findall(str(tag.contents)):
            line_station+=[line_pattern.findall(str(tag.contents))[0][1]]   #部分网页tag.string返回空值
    #网页信息错误
    if line_name[-4:-2]=='17': line_station[0]='虹桥火车站' #17号线站点名缺失
    if line_name[-3]=='4': line_station[18]='上海火车站' #4号线站点名缺失
    if line_name[-4:-2]=='10':line_station=[line_station[i] for i in range(len(line_station)) if i<=len(line_station)-7] #10号记入未开通站点
    if line_name[-3]=='5':line_station.insert(-8,'奉浦大道')  #5号奉浦大道链接缺失
    for i in range(len(line_station)-1):
        if not station_connection[line_station[i]]:
            station_connection[line_station[i]]=[[line_station[i+1],line_name[4:]]]
        elif station_connection[line_station[i]] and ([line_station[i+1],line_name[4:]] not in station_connection[line_station[i]]):
            station_connection[line_station[i]].append([line_station[i+1],line_name[4:]])
        if not station_connection[line_station[i+1]]:
            station_connection[line_station[i+1]]=[[line_station[i],line_name[4:]]]
        elif station_connection[line_station[i+1]] and ([line_station[i],line_name[4:]] not in station_connection[line_station[i+1]]):
            station_connection[line_station[i+1]].append([line_station[i],line_name[4:]])
    #特殊路线修正
    if line_name[-3]=='4':station_connection['宜山路'].append(['虹桥路','4号线']) #4号为环线
    if line_name[-4:-2]=='11':   #11号有2条支线
        station_connection['嘉定新城'].append(['上海赛车场','11号线'])
        station_connection['上海赛车场'].append(['嘉定新城','11号线'])
        station_connection['嘉定北'].remove(['上海赛车场','11号线'])
        station_connection['上海赛车场'].remove(['嘉定北','11号线'])
    if line_name[-4:-2]=='10':    #10号有2条支线
        station_connection['龙柏新村'].remove(['虹桥火车站','10号线'])
        station_connection['虹桥火车站'].remove(['龙柏新村','10号线'])
        station_connection['龙溪路'].append(['龙柏新村','10号线'])
        station_connection['龙柏新村'].append(['龙溪路','10号线'])
    if line_name[-3]=='5':    #5号有2条支线
        station_connection['奉贤新城'].remove(['金平路','5号线'])
        station_connection['金平路'].remove(['奉贤新城','5号线'])
        station_connection['东川路'].append(['金平路','5号线'])
        station_connection['金平路'].append(['东川路','5号线'])
station_connection

defaultdict(list,
            {'七宝': [['星中路', '9号线'], ['中春路', '9号线']],
             '七莘路': [['虹莘路', '12号线']],
             '三林': [['东方体育中心', '11号线'], ['三林东', '11号线']],
             '三林东': [['三林', '11号线'], ['浦三路', '11号线']],
             '三门路': [['江湾体育场', '10号线'], ['殷高东路', '10号线']],
             '三鲁公路': [['沈杜公路', '浦江线'], ['闵瑞路', '浦江线']],
             '上南路': [['灵岩南路', '6号线'], ['华夏西路', '6号线']],
             '上大路': [['南陈路', '7号线'], ['场中路', '7号线']],
             '上海体育场': [['上海体育馆', '4号线'], ['东安路', '4号线']],
             '上海体育馆': [['漕宝路', '1号线'],
              ['徐家汇', '1号线'],
              ['宜山路', '4号线'],
              ['上海体育场', '4号线']],
             '上海儿童医学中心': [['临沂新村', '6号线'], ['蓝村路', '6号线']],
             '上海动物园': [['虹桥1号航站楼', '10号线'], ['龙溪路', '10号线']],
             '上海南站': [['锦江乐园', '1号线'], ['漕宝路', '1号线'], ['石龙路', '3号线']],
             '上海图书馆': [['交通大学', '10号线'], ['陕西南路', '10号线']],
             '上海大学': [['祁华路', '7号线'], ['南陈路', '7号线']],
             '上海汽车城': [['安亭', '11号线'], ['昌吉东路', '11号线

In [3]:
#Latitude and longitude info of each station extracted by using Baidu API. To avoid visiting too frequently, save it as string.
locationstring="""
'七宝': (31.161393, 121.356049),
'七莘路': (31.137623999999999, 121.36970600000001),
'三林': (31.148688, 121.517556),
'三林东': (31.152358000000003, 121.52967099999999),
'三门路': (31.317294, 121.49271599999999),
'三鲁公路': (31.128497999999997, 121.513682),
'上南路': (31.156859999999998, 121.51181200000001),
'上大路': (31.321458000000003, 121.41477500000001),
'上海体育场': (31.188157, 121.44855099999999),
'上海体育馆': (31.188814000000001, 121.44340200000001),
'上海儿童医学中心': (31.211585999999997, 121.52927700000001),
'上海动物园': (31.199022999999997, 121.36928400000001),
'上海南站': (31.159439000000003, 121.43586499999999),
'上海图书馆': (31.213840999999999, 121.451117),
'上海大学': (31.321440000000003, 121.39903000000001),
'上海汽车城': (31.291581000000001, 121.18725000000001),
'上海游泳馆': (31.185365000000001, 121.448026),
'上海火车站': (31.255922999999999, 121.462056),
'上海科技馆': (31.224356, 121.54782299999999),
'上海西站': (31.268790000000003, 121.409305),
'上海赛车场': (31.337944, 121.232625),
'上海马戏城': (31.285845000000002, 121.45850300000001),
'下南路': (31.183753000000003, 121.54431699999999),
'世博会博物馆': (31.200208, 121.48881000000002),
'世博大道': (31.188230999999998, 121.49093700000002),
'世纪公园': (31.222046000000002, 121.55897900000001),
'世纪大道': (31.234809999999996, 121.533534),
'东兰路': (31.161428999999998, 121.39854299999999),
'东城一路': (31.036318999999999, 121.53870800000001),
'东安路': (31.197303999999999, 121.461153),
'东宝兴路': (31.265746000000004, 121.486639),
'东川路': (31.024465000000003, 121.42632900000001),
'东方体育中心': (31.159351000000001, 121.48701200000001),
'东方绿舟': (31.114696999999996, 121.01713899999999),
'东昌路': (31.239078999999997, 121.521998),
'东明路': (31.178411999999998, 121.51798400000001),
'东陆路': (31.288630999999999, 121.58578300000001),
'东靖路': (31.296646999999997, 121.595386),
'中兴路': (31.266234999999998, 121.48454099999999),
'中华艺术宫': (31.190004999999999, 121.50131499999999),
'中山公园': (31.227830999999998, 121.42551100000001),
'中山北路': (31.265274999999999, 121.465616),
'中春路': (31.155953999999998, 121.34474299999999),
'中潭路': (31.260932, 121.44746699999999),
'中科路': (31.185365000000001, 121.613522),
'丰庄': (31.248388000000002, 121.361851),
'临平路': (31.266579999999998, 121.507604),
'临沂新村': (31.198789000000001, 121.52355),
'临港大道': (30.929828999999998, 121.91725500000001),
'九亭': (31.143605999999998, 121.32593),
'书院': (30.965002000000002, 121.85722),
'云台路': (31.187778999999999, 121.506924),
'云山路': (31.256678000000004, 121.57916499999999),
'云锦路': (31.172650000000001, 121.46491399999999),
'五洲大道': (31.308621999999996, 121.59588899999999),
'五莲路': (31.278007000000002, 121.59453999999999),
'五角场': (31.303805000000001, 121.52119099999999),
'交通大学': (31.208553999999999, 121.440977),
'人民广场': (31.238795, 121.481022),
'伊犁路': (31.204634000000002, 121.41067099999999),
'佘山': (31.101348999999999, 121.201875),
'兆丰路': (31.294652000000003, 121.156828),
'光明路': (31.336686, 121.62553999999999),
'共富新村': (31.357246999999997, 121.43940800000001),
'共康路': (31.325354999999998, 121.453464),
'凌兆新村': (31.146941999999999, 121.49616),
'凌空路': (31.198565000000002, 121.730672),
'刘行': (31.363427000000001, 121.368889),
'创新中路': (31.21988, 121.680196),
'剑川路': (31.013750000000002, 121.36223200000001),
'北新泾': (31.222103999999998, 121.38063999999999),
'北桥': (31.043566999999999, 121.40558500000002),
'北洋泾路': (31.245588000000001, 121.559229),
'北蔡': (31.186466999999997, 121.55876299999998),
'华夏东路': (31.202995000000001, 121.687321),
'华夏中路': (31.181362, 121.58883600000001),
'华夏西路': (31.155549000000001, 121.52124999999999),
'华宁路': (31.013115999999997, 121.40176599999999),
'华鹏路': (31.183508000000003, 121.53110500000001),
'南京东路': (31.244605, 121.495976),
'南京西路': (31.23413, 121.46593200000001),
'南浦大桥': (31.210031000000001, 121.51478600000002),
'南翔': (31.303357000000002, 121.329747),
'南陈路': (31.327223999999998, 121.405224),
'博兴路': (31.270058000000002, 121.593414),
'友谊西路': (31.387466999999997, 121.43444),
'友谊路': (31.392360999999998, 121.441926),
'台儿庄路': (31.260303000000004, 121.604474),
'合川路': (31.172140999999996, 121.39126200000001),
'同济大学': (31.289027000000001, 121.508532),
'后滩': (31.177884000000002, 121.48003500000002),
'周浦东': (31.115609999999997, 121.61360900000001),
'呼兰路': (31.346101000000001, 121.444158),
'唐镇': (31.214227000000001, 121.66154399999999),
'商城路': (31.236125000000001, 121.52288999999999),
'嘉善路': (31.208615999999996, 121.467353),
'嘉定北': (31.397409000000003, 121.24401),
'嘉定新城': (31.335744000000002, 121.26103999999999),
'嘉定西': (31.383236, 121.234404),
'嘉松中路': (31.178840999999998, 121.231058),
'四川北路': (31.269299, 121.490582),
'四平路': (31.280539000000001, 121.50803700000002),
'国权路': (31.295034000000001, 121.51676699999999),
'国际客运中心': (31.255834999999998, 121.504856),
'场中路': (31.308927000000001, 121.42482800000001),
'塘桥': (31.215406999999999, 121.52535300000001),
'复兴岛': (31.287179999999999, 121.56783899999999),
'外环路': (31.126655, 121.399619),
'外高桥保税区北': (31.353854999999999, 121.593469),
'外高桥保税区南': (31.327459999999999, 121.60853999999999),
'大世界': (31.233128000000004, 121.48588500000001),
'大华三路': (31.280185999999997, 121.42948799999999),
'大场镇': (31.293054999999999, 121.423427),
'大木桥路': (31.200221999999997, 121.47016599999999),
'大柏树': (31.295206, 121.489755),
'大渡河路': (31.237534999999998, 121.401707),
'大连路': (31.263959000000003, 121.519428),
'天潼路': (31.250088000000002, 121.48878300000001),
'奉浦大道': (30.952282, 121.478903),
'奉贤新城': (30.923722999999999, 121.50965500000001),
'威宁路': (31.220484000000003, 121.393672),
'娄山关路': (31.216958000000002, 121.41081799999999),
'嫩江路': (31.320922999999997, 121.53855900000001),
'学林路': (31.189110999999997, 121.62100000000001),
'宁国路': (31.274518, 121.53891200000001),
'安亭': (31.294334999999997, 121.16860200000001),
'宋园路': (31.202507000000001, 121.418451),
'宜山路': (31.192762999999999, 121.433677),
'宝安公路': (31.327171999999997, 121.256621),
'宝山路': (31.257393, 121.48291),
'宝杨路': (31.388085999999998, 121.45936100000002),
'富锦路': (31.398541999999999, 121.43108700000001),
'小南门': (31.222656000000001, 121.50489499999999),
'岚皋路': (31.262583000000003, 121.42833899999999),
'川沙': (31.192885999999998, 121.704499),
'巨峰路': (31.286325000000001, 121.59553500000001),
'市光路': (31.328515999999997, 121.538527),
'常熟路': (31.219964000000001, 121.45662800000001),
'广兰路': (31.219628999999998, 121.624105),
'康新公路': (31.136032, 121.62395500000001),
'延吉中路': (31.294723999999999, 121.54140500000001),
'延安西路': (31.215771999999998, 121.42357700000001),
'延长路': (31.278059999999996, 121.46189299999999),
'张华浜': (31.363646000000003, 121.505376),
'张江路': (31.194945000000001, 121.635521),
'张江高科': (31.207785999999999, 121.59419399999999),
'彭浦新村': (31.314134999999997, 121.457393),
'徐家汇': (31.201202000000002, 121.442314),
'徐泾东': (31.194437000000001, 121.305802),
'徐泾北城': (31.181221999999998, 121.24823300000001),
'徐盈路': (31.183965999999998, 121.26062399999999),
'御桥': (31.164484999999999, 121.57710800000001),
'德平路': (31.251721999999997, 121.570733),
'惠南': (31.059628999999997, 121.76826499999999),
'惠南东': (31.032830999999998, 121.800265),
'成山路': (31.176614000000001, 121.50338700000002),
'打浦桥': (31.212072999999997, 121.475402),
'提篮桥': (31.259140999999996, 121.51328000000001),
'文井路': (31.009176, 121.38743700000001),
'新场': (31.051514000000001, 121.65556100000001),
'新天地': (31.221997999999999, 121.48168600000001),
'新村路': (31.270157000000001, 121.429033),
'新江湾城': (31.334451000000001, 121.51318799999999),
'新闸路': (31.243725000000001, 121.46858200000001),
'昌吉东路': (31.299940000000003, 121.206771),
'昌平路': (31.240162999999999, 121.44903899999998),
'星中路': (31.163861999999998, 121.37557099999999),
'春申路': (31.103958000000002, 121.39233200000001),
'曲阜路': (31.248287999999999, 121.47803600000002),
'曲阳路': (31.282156000000001, 121.49733300000001),
'曹杨路': (31.244676000000002, 121.42441100000001),
'曹路': (31.277549, 121.689449),
'望园路': (30.905503999999997, 121.501),
'朱家角': (31.116640000000004, 121.06083799999999),
'李子园': (31.274713000000002, 121.396607),
'杨思': (31.166709999999998, 121.50028999999999),
'杨树浦路': (31.277401000000001, 121.562901),
'杨高中路': (31.233821000000002, 121.55514699999999),
'杨高北路': (31.285853000000003, 121.60942299999999),
'杨高南路': (31.193346000000002, 121.531902),
'松江体育中心': (31.022047999999998, 121.23708899999998),
'松江南站': (30.989169, 121.23673700000001),
'松江大学城': (31.059718, 121.238922),
'松江新城': (31.036209000000003, 121.23725900000001),
'枫桥路': (31.248068, 121.41753999999999),
'桂林公园': (31.171505, 121.42437199999999),
'桂林路': (31.181001000000002, 121.424982),
'桃浦新村': (31.287646999999996, 121.35593700000001),
'武威路': (31.282824999999999, 121.37131799999999),
'武宁路': (31.250646999999997, 121.408597),
'殷高东路': (31.327539000000002, 121.51356200000001),
'殷高西路': (31.325554999999998, 121.49148000000001),
'民生路': (31.240963000000001, 121.55161799999999),
'民雷路': (31.274649, 121.674609),
'水产路': (31.387091999999999, 121.494702),
'水城路': (31.205074, 121.398444),
'汇臻路': (31.031161999999998, 121.53118500000001),
'汇金路': (31.166940000000004, 121.15833500000001),
'汉中路': (31.247271999999999, 121.46513400000001),
'江宁路': (31.250707000000002, 121.45103),
'江川路': (31.004953000000004, 121.39646699999999),
'江月路': (31.089984999999999, 121.515208),
'江杨北路': (31.414233000000003, 121.44623500000002),
'江浦公园': (31.271843000000001, 121.530635),
'江浦路': (31.280818, 121.524804),
'江湾体育场': (31.312296000000003, 121.523234),
'江湾镇': (31.311153000000004, 121.49163600000001),
'江苏路': (31.226557, 121.43771399999999),
'汶水路': (31.298973999999998, 121.45654099999999),
'沈杜公路': (31.067057000000002, 121.518849),
'泗泾': (31.123981000000001, 121.26698200000001),
'洞泾': (31.090390999999997, 121.23703799999998),
'洲海路': (31.318355, 121.595922),
'浦三路': (31.157147999999999, 121.54553999999999),
'浦东国际机场': (31.157478000000001, 121.81508999999998),
'浦东大道': (31.277652000000003, 121.58242800000001),
'浦江镇': (31.081471999999998, 121.52304599999999),
'浦电路': (31.229006999999999, 121.53851299999999),
'浦航路': (31.049126000000001, 121.54794299999999),
'海伦路': (31.264853999999996, 121.495621),
'海天三路': (31.174865999999998, 121.803326),
'淀山湖大道': (31.140734000000002, 121.08848300000001),
'淞发路': (31.350777000000001, 121.507047),
'淞滨路': (31.376563000000001, 121.49938700000001),
'淞虹路': (31.223953999999999, 121.36601100000001),
'淮海中路': (31.209091999999998, 121.44247900000001),
'港城路': (31.359262000000001, 121.58131000000002),
'源深体育中心': (31.237807, 121.543447),
'滴水湖': (30.904731999999999, 121.93713200000001),
'漕宝路': (31.164833000000002, 121.39955),
'漕河泾开发区': (31.176220000000001, 121.40416399999999),
'漕溪路': (31.182978000000002, 121.44473899999998),
'漕盈路': (31.166647999999999, 121.10271000000002),
'潘广路': (31.370197999999998, 121.36229299999999),
'灵岩南路': (31.154358000000002, 121.501797),
'爱国路': (31.286351, 121.559427),
'环城东路': (30.937138000000001, 121.470082),
'申江路': (31.248959000000003, 121.631952),
'白银路': (31.351371, 121.251801),
'真北路': (31.237860999999999, 121.38888700000001),
'真如': (31.256496999999996, 121.41383400000001),
'石龙路': (31.164303999999998, 121.44963),
'祁华路': (31.328108, 121.38016),
'祁连山南路': (31.243220000000001, 121.373949),
'祁连山路': (31.277204999999999, 121.382499),
'秀沿路': (31.143946000000003, 121.60505800000001),
'紫藤路': (31.175533000000001, 121.37097),
'罗南新村': (31.394509999999997, 121.364047),
'罗山路': (31.159351000000001, 121.59962400000001),
'美兰湖': (31.407853000000003, 121.356426),
'翔殷路': (31.313445000000002, 121.552049),
'耀华路': (31.184180999999999, 121.50115600000001),
'老西门': (31.224798, 121.49008000000001),
'联航路': (31.079252, 121.51721200000001),
'肇嘉浜路': (31.205392, 121.45646299999999),
'自然博物馆': (31.241120000000002, 121.46915200000001),
'航中路': (31.171346999999997, 121.36145),
'航头东': (31.060571999999997, 121.62411499999999),
'航津路': (31.341304999999998, 121.60053899999998),
'芦恒路': (31.124776000000001, 121.504768),
'花木路': (31.217570000000002, 121.569227),
'花桥': (31.304845, 121.11070600000001),
'芳华路': (31.199404999999999, 121.55651899999999),
'芳甸路': (31.237703999999997, 121.56461999999999),
'莘庄': (31.116871999999997, 121.39218600000001),
'莲溪路': (31.182341999999998, 121.569428),
'莲花路': (31.136734000000004, 121.40933400000002),
'萧塘': (30.972265999999998, 121.448398),
'蓝天路': (31.247173999999998, 121.58429599999999),
'蓝村路': (31.217627, 121.53438100000001),
'虹口足球场': (31.277218999999999, 121.48725900000001),
'虹桥1号航站楼': (31.197604999999999, 121.35368999999999),
'虹桥2号航站楼': (31.200616999999998, 121.3327),
'虹桥火车站': (31.200546999999997, 121.32699699999999),
'虹桥路': (31.203176000000003, 121.42804299999999),
'虹梅路': (31.166072999999997, 121.40393200000001),
'虹漕路': (31.170192, 121.41703100000001),
'虹莘路': (31.149835999999997, 121.387405),
'蟠龙路': (31.192485999999999, 121.28513799999999),
'行知路': (31.291225000000001, 121.428059),
'衡山路': (31.213390000000004, 121.45472600000001),
'西渡': (30.996169000000002, 121.44015400000001),
'西藏北路': (31.269400000000001, 121.47525),
'西藏南路': (31.221940999999998, 121.490982),
'诸光路': (31.197876000000001, 121.29964099999999),
'豫园': (31.232713, 121.49873100000001),
'赤峰路': (31.286968999999999, 121.488997),
'赵巷': (31.167509999999996, 121.198705),
'远东大道': (31.205140999999998, 121.76203000000001),
'迪士尼': (31.148266999999997, 121.67196399999999),
'通河新村': (31.337451000000001, 121.447965),
'邮电新村': (31.276892999999998, 121.50236799999999),
'醉白池': (31.007759999999998, 121.23751599999999),
'野生动物园': (31.06138, 121.72799499999999),
'金京路': (31.285413000000002, 121.62220500000001),
'金吉路': (31.270009000000002, 121.635116),
'金平路': (31.017171999999999, 121.41654699999999),
'金桥': (31.266301000000002, 121.617822),
'金桥路': (31.263276000000001, 121.58833700000001),
'金沙江西路': (31.247314000000003, 121.34104099999999),
'金沙江路': (31.248087999999999, 121.36328799999998),
'金海湖': (40.187348, 117.310896),
'金海路': (31.269219, 121.64533200000001),
'金科路': (31.209932000000002, 121.608557),
'金运路': (31.247367999999998, 121.326035),
'铁力路': (31.414309000000003, 121.46784199999999),
'银都路': (31.090915000000003, 121.38412700000001),
'锦江乐园': (31.145541999999999, 121.415479),
'锦绣路': (31.193648, 121.54655200000001),
'镇坪路': (31.252818999999999, 121.43686399999999),
'长寿路': (31.246717999999998, 121.44476200000001),
'长江南路': (31.337846999999996, 121.49818),
'长清路': (31.180329999999998, 121.492645),
'闵瑞路': (31.054121000000002, 121.536901),
'闵行开发区': (31.006271999999996, 121.37650900000001),
'陆家嘴': (31.243811999999998, 121.50898400000001),
'陆家浜路': (31.217388, 121.49269099999999),
'陈春路': (31.177982, 121.553832),
'陕西南路': (31.221939000000003, 121.465323),
'隆德路': (31.236733000000001, 121.430026),
'隆昌路': (31.281347999999998, 121.550831),
'青浦新城': (31.164546999999999, 121.13190300000001),
'静安寺': (31.229810999999998, 121.45167099999999),
'鞍山新村': (31.278921999999998, 121.516066),
'顾唐路': (31.272153000000003, 121.66308899999999),
'顾戴路': (31.128240000000002, 121.35008400000001),
'顾村公园': (31.346915000000003, 121.38368899999999),
'颛桥': (31.071874999999999, 121.407141),
'马当路': (31.215626, 121.48338500000001),
'马陆': (31.325402, 121.28343000000001),
'高科西路': (31.191403000000001, 121.516553),
'高青路': (31.165436, 121.522143),
'鲁班路': (31.204940000000001, 121.48098799999998),
'鹤沙航城': (31.083657000000002, 121.61785),
'黄兴公园': (31.299441999999999, 121.53642499999999),
'黄兴路': (31.302414000000002, 121.52385200000001),
'黄陂南路': (31.228753000000001, 121.479798),
'龙华': (31.179259000000002, 121.45922900000001),
'龙华中路': (31.190940000000001, 121.46334399999999),
'龙柏新村': (31.183294, 121.37692800000001),
'龙溪路': (31.200046999999998, 121.38674099999999),
'龙漕路': (31.176190999999996, 121.45043000000001),
'龙耀路': (31.165971999999996, 121.466196),
'龙阳路': (31.209165999999996, 121.56404999999999)
"""

In [4]:
#Turn the location info into dict
station_location=defaultdict(list)
for item in locationstring.split('\n'):
    if item:
        station=item.split(':')[0][1:-1]
        lat=item.split(':')[1].split(',')[0][2:]
        lon=item.split(':')[1].split(',')[1][1:-1]
        lat,lon=float(lat),float(lon)
        station_location[station]=(lat,lon)
station_location       

defaultdict(list,
            {'七宝': (31.161393, 121.356049),
             '七莘路': (31.137624, 121.36970600000001),
             '三林': (31.148688, 121.517556),
             '三林东': (31.152358000000003, 121.529671),
             '三门路': (31.317294, 121.49271599999999),
             '三鲁公路': (31.128497999999997, 121.513682),
             '上南路': (31.156859999999998, 121.511812),
             '上大路': (31.321458000000003, 121.414775),
             '上海体育场': (31.188157, 121.448551),
             '上海体育馆': (31.188814, 121.443402),
             '上海儿童医学中心': (31.211585999999997, 121.52927700000001),
             '上海动物园': (31.199022999999997, 121.36928400000001),
             '上海南站': (31.159439000000003, 121.43586499999999),
             '上海图书馆': (31.213841, 121.451117),
             '上海大学': (31.321440000000003, 121.39903000000001),
             '上海汽车城': (31.291581, 121.18725),
             '上海游泳馆': (31.185365, 121.448026),
             '上海火车站': (31.255923, 121.462056),
             '上海科技馆': (31.224356,

### Uninformed Search

#### (1) BFS

In [5]:
# BFS with search strategy(alterable step cost)
def standard_bfs(graph,start,destination,search_strategy):
    visited=set()
    transfer=set()
    pathes=[[[start],transfer]]
    while pathes:
        path_wtrans=pathes.pop(0)
        path=path_wtrans[0]
        transfer=path_wtrans[1]
        frontier=path[-1]
        if frontier in visited:continue
        successors=graph[frontier]
        for station in successors:
            if station[0] in path: continue
            new_trans=transfer|{station[1]}
            new_path=path+[station[0]]
            if station[0]==destination:
                return 'path found with {} stations: {} with number of transfers: {}'.format(len(new_path)-1,new_path,len(new_trans)-1)
            pathes.append([new_path,new_trans]) 
        pathes=search_strategy(pathes)
        visited.add(frontier)

In [55]:
%%time
standard_bfs(station_connection,'新天地','源深体育中心',search_strategy=lambda x:x)

Wall time: 0 ns


"path found with 6 stations: ['新天地', '老西门', '陆家浜路', '小南门', '商城路', '世纪大道', '源深体育中心'] with number of transfers: 3"

##### BFS+Incurred Cost Function

##### (a) BFS+cost combining number of stations & transfers

In [56]:
def sort_by_combined_cost(pathes,transfer_cost=2):  # Given the time taken during transfer, assume it costs 2 station per transfer
    def combined_cost(path):
        return (len(path[0])-1)+(len(path[1])-1)*transfer_cost
    return sorted(pathes,key=combined_cost)
standard_bfs(station_connection,'新天地','源深体育中心',search_strategy=sort_by_combined_cost)

"path found with 7 stations: ['新天地', '老西门', '豫园', '南京东路', '陆家嘴', '东昌路', '世纪大道', '源深体育中心'] with number of transfers: 2"

##### (b) BFS+shortest path(assuming identical costs between 2 stations, i.e. minimum number of stations)

In [57]:
def sort_by_combined_cost(pathes,transfer_cost=0):  #Transfer has no impact on shortest path choice
    def combined_cost(path):
        return (len(path[0])-1)+(len(path[1])-1)*transfer_cost
    return sorted(pathes,key=combined_cost)
standard_bfs(station_connection,'新天地','源深体育中心',search_strategy=sort_by_combined_cost)

"path found with 6 stations: ['新天地', '老西门', '陆家浜路', '小南门', '商城路', '世纪大道', '源深体育中心'] with number of transfers: 3"

##### (c) BFS+minimum transfers

In [59]:
def sort_by_combined_cost(pathes,transfer_cost=10000):  #Transfer is the critical factor, put a large number for its cost
    def combined_cost(path):
        return (len(path[0])-1)+(len(path[1])-1)*transfer_cost
    return sorted(pathes,key=combined_cost)
standard_bfs(station_connection,'新天地','源深体育中心',search_strategy=sort_by_combined_cost)

"path found with 7 stations: ['新天地', '老西门', '豫园', '南京东路', '陆家嘴', '东昌路', '世纪大道', '源深体育中心'] with number of transfers: 2"

#### (2) DFS

In [10]:
# DFS with search strategy(alterable step cost)
def standard_dfs(graph,start,destination,search_strategy):
    visited=set()
    transfer=set()
    pathes=[[[start],transfer]]
    while pathes:
        path_wtrans=pathes.pop(0)
        path=path_wtrans[0]
        transfer=path_wtrans[1]
        frontier=path[-1]
        if frontier in visited: continue
        successors=graph[frontier]
        for station in successors:
            if station[0] in path: continue
            new_trans=transfer|{station[1]}
            new_path=path+[station[0]]
            if station[0]==destination:
                return 'path found with {} stations: {} with number of transfers: {}'.format(len(new_path)-1,new_path,len(new_trans)-1)
            pathes=[[new_path,new_trans]]+pathes
        pathes=search_strategy(pathes)
        visited.add(frontier)

In [60]:
%%time
standard_dfs(station_connection,'新天地','源深体育中心',search_strategy=lambda x:x)

Wall time: 0 ns


"path found with 26 stations: ['新天地', '马当路', '世博会博物馆', '世博大道', '长清路', '成山路', '东明路', '华鹏路', '下南路', '北蔡', '陈春路', '莲溪路', '华夏中路', '龙阳路', '芳华路', '锦绣路', '杨高南路', '高科西路', '云台路', '耀华路', '中华艺术宫', '西藏南路', '陆家浜路', '小南门', '商城路', '世纪大道', '源深体育中心'] with number of transfers: 5"

#### (3) Iterative Deepening DFS

In [53]:
# Iterative DFS with identical step costs
class Node_DFS:
    def __init__(self, state, line):
        self.state=state
        self.line=line

def Iterative_DFS(graph,start,destination): #返回3种结果:最优路径/站点名错误返回failure/当前深度限制下无法找到最优站点返回path not found
    for i in range(1,50):
        result = standard_DLS(graph,start,destination,i)
        if result[0][0]=='cutoff': continue
        elif result[0][0] =='failure': return 'Station Name Error'
        else:
            return 'path found with {} stations: {} with number of transfers: {}'.format(len(result[0])-1,result[0],len(result[1])-1)
    return 'Path not found within the current limit'
                
def standard_DLS(graph,start,destination,limit):
    if (start not in [key for key in graph.keys()]) or (destination not in [key for key in graph.keys()]):
        return ['failure'],set()
    else:
        #global visited   不能增加visited，否则上条达到cutoff路径中的站点不会出现在后续路径中，导致无解
        #visited=[]
        node=Node_DFS(start,None)
        result=recursive_DLS(graph,node,destination,limit)
        path=[node if node in ['cutoff','failure'] else node.state for node in result ]
        transfer=set([node.line for node in result if (node not in ['cutoff','failure'] and node.line is not None)])
        return path, transfer
    
def recursive_DLS(graph,node,destination,limit):  #在limit限制内走完所有可能路径，直至找到到达终点的路径，路径内node可能重复
    #global visited                               #返回3种结果:到达终点的路径/limit前无法找到路径返回cutoff/站点名不存在返回failure 
    #visited.append(node.state)
    if node.state==destination:return [node]
    elif limit==0:return ['cutoff']
    elif node.state not in [key for key in graph.keys()]: return ['failure']
    else:
        cutoff_occurred=False
        for station in graph[node.state]:
            child=Node_DFS(station[0],station[1])
            #if child.state in visited: continue
            result=[node]+recursive_DLS(graph,child,destination,limit-1)
            if result[-1]=='cutoff': 
                cutoff_occurred=True
            elif result[-1]!='failure': return result
        if cutoff_occurred: return ['cutoff']
        else: return ['failure']

In [61]:
%%time
Iterative_DFS(station_connection,'新天地','源深体育中心')

Wall time: 14 ms


"path found with 6 stations: ['新天地', '老西门', '陆家浜路', '小南门', '商城路', '世纪大道', '源深体育中心'] with number of transfers: 3"

#### (4) Bidirectional search

In [63]:
#Bidirectional search with identical step costs
#According to Chap3, some additional search is required to make sure if there's another short-cut as the first path may not be optimal.
#When trying to generate and check frontier one by one, the solution is not optimal, so the below generates frontiers with same depth sumultanaously to make sure the solution has shortest length.
#Nodes kept in memory are the worst case for bidirectional search 
def path_combine(path_s,path_d):   #路径重合时汇总路径
    result=path_s[:-1]+[path_d[len(path_d)-i-1] for i in range(0,len(path_d))]
    result_path=[node.state for node in result]
    result_trans=set([node.line for node in result if node.line is not None])
    return 'path found with {} stations: {} with number of transfers: {}'.format(len(result_path)-1,result_path,len(result_trans)-1)

def node_generation(graph,pathes):   #批量生成深度+1的路径
    new_pathes=[]
    list_frontier=[path[-1].state for path in pathes]
    for i in range(len(list_frontier)):
        successors=graph[list_frontier[i]]
        for station in successors:
            node=Node_DFS(station[0],station[1])
            if node in pathes[i]:continue
            new_path=pathes[i]+[node]
            new_pathes.append(new_path)
    return new_pathes

def frontier_overlap(pathes,pathes_reverse):   #比较路径间有无交集
    frontier=[path[-1].state for path in pathes]
    frontier_reverse=[path[-1].state for path in pathes_reverse]
    frontier_both=[x for x in frontier if x in frontier_reverse]
    if frontier_both:
        frontier_index=frontier.index(frontier_both[0])
        frontier_reverse_index=frontier_reverse.index(frontier_both[0])
        return path_combine(pathes[frontier_index],pathes_reverse[frontier_reverse_index])
    else: return []
    
def bidirection(graph,start,destination):
    node_s=Node_DFS(start,None)
    node_d=Node_DFS(destination,None)
    pathes_s=[[node_s]]
    pathes_d=[[node_d]]
    while pathes_s and pathes_d:
        result=frontier_overlap(pathes_s,pathes_d)
        if result: return result
        new_pathes_s=node_generation(graph,pathes_s)
        new_pathes_d=node_generation(graph,pathes_d)
        result=frontier_overlap(new_pathes_s,pathes_d)
        if result: return result
        result=frontier_overlap(pathes_s,new_pathes_d)
        if result: return result
        pathes_s=new_pathes_s
        pathes_d=new_pathes_d
    return 'No path found'

In [64]:
%%time
bidirection(station_connection,'新天地','源深体育中心')

Wall time: 1e+03 µs


"path found with 6 stations: ['新天地', '老西门', '陆家浜路', '小南门', '商城路', '世纪大道', '源深体育中心'] with number of transfers: 3"

### Informed Search:Find the solution more efficiently

#### (1) Greedy best-first search

##### Cost funciton purely decided by estimated cost

In [72]:
import math
class Node_Informed:
    def __init__(self,state,line,esti_cost):
        self.state=state
        self.line=line
        self.esti_cost=esti_cost
def geo_distance(origin,destination):  
#haversine distance基于经纬度的距离计算
#haversin(d/R)=haversine(lat2-lat1)+cos(lat1)cos(lat2)haversine(long2-long1)
#haversine(x)=sin(x/2)^2=(1-cos(x))/2
#R为地球半径，取均值6371km
    lat1,lon1=origin
    lat2,lon2=destination
    radius=6371
    dlat=math.radians(lat2-lat1)   #radians为角度转弧度
    dlon=math.radians(lon2-lon1)
    a=((math.sin(dlat/2))**2)+math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*((math.sin(dlon/2))**2)
    d=radius*math.asin(math.sqrt(a))*2
    return d
def estimate_distance(station,destination):
    return geo_distance(station_location[station],station_location[destination])
def greedy_bfs(graph,start,destination,search_strategy):
    node=Node_Informed(start,None,estimate_distance(start,destination))
    visited=set()
    pathes=[[node]]
    while pathes:
        path=pathes.pop(0)
        frontier=path[-1]
        if frontier.state in visited:continue
        successors=graph[frontier.state]
        for station in successors:
            node=Node_Informed(station[0],station[1],estimate_distance(station[0],destination))
            if node in path: continue
            new_path=path+[node]
            if node.state==destination:
                new_path_trans=set([node.line for node in new_path if node.line is not None])
                new_path_path=[node.state for node in new_path]
                return 'path found with {} stations: {} with number of transfers: {}'.format(len(new_path)-1,new_path_path,len(new_path_trans)-1)
            pathes.append(new_path) 
        pathes=sorted(pathes,key=search_strategy)
        visited.add(frontier.state)

In [73]:
%%time
greedy_bfs(station_connection,'新天地','源深体育中心',search_strategy=lambda x: x[-1].esti_cost)

Wall time: 0 ns


"path found with 7 stations: ['新天地', '老西门', '豫园', '南京东路', '陆家嘴', '东昌路', '世纪大道', '源深体育中心'] with number of transfers: 2"

#### (2) A* Search (only change ranking key for greedy best first)

##### Cost funciton equal to incurred cost+estimated cost
##### Assume the distance between 2 stations is about 1.2km, total cost=incurred cost(#stations)*1.2+estimated cost(#km)

##### Without considering transfer cost

In [74]:
%%time
greedy_bfs(station_connection,'新天地','源深体育中心',search_strategy=lambda x: (len(x)-1)*1.2+x[-1].esti_cost)

Wall time: 1 ms


"path found with 6 stations: ['新天地', '老西门', '陆家浜路', '小南门', '商城路', '世纪大道', '源深体育中心'] with number of transfers: 3"

##### Considering transfer cost (set transfer cost as 2 more stations)

In [75]:
%%time
greedy_bfs(station_connection,'新天地','源深体育中心',search_strategy=lambda x: (len(x)-1+(len(set([item.line for item in x if item.line is not None]))-1)*2)*1.2+x[-1].esti_cost)

Wall time: 1e+03 µs


"path found with 7 stations: ['新天地', '老西门', '豫园', '南京东路', '陆家嘴', '东昌路', '世纪大道', '源深体育中心'] with number of transfers: 2"